In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [4]:
# Crear carpeta "images" si no existe
if not os.path.exists("images"):
    os.makedirs("images")

In [ ]:
pd.set_option('display.max_columns', None) # es una configuración de la librería Pandas que controla cómo se visualizan los DataFrames cuando los imprimes en la pantalla

# 'display.max_columns' le dice a Pandas que estás configurando el número máximo de columnas que debe mostrar cuando imprime un DataFrame.
# El Valor: None Al establecer el valor en None, le estás diciendo a Pandas que desactive el límite.
# Quiero que me muestres TODAS las columnas de mi DataFrame, sin importar cuántas tenga, en lugar de ocultar las del medio y poner puntos suspensivos (...)."
# Es una herramienta esencial para el Análisis Exploratorio de Datos (EDA), ya que te ayuda a ver la "foto completa" de tu conjunto de datos

## Carga y exploración inicial:

In [ ]:
# En este caso no es adecuado usar el index_col = 0, porque para la función merge que tendré que usar para unir los csv, necesito que 'loyalty number' sea una columna regular
df_history = pd.read_csv("data/Customer Loyalty History.csv") 
#primera exploración visual
df_history

In [ ]:
df_activity = pd.read_csv("data/Customer Flight Activity.csv")
df_activity

In [6]:
def archivar_grafica(nombre):
    ruta = f"images/{nombre}.png"
    plt.savefig(ruta, dpi=300, bbox_inches="tight")
    print(f" Gráfica archivada en: {ruta}")

In [ ]:
# visual rápida de los 2 df: 

df_history

## Función localizar duplicados en df's:


In [ ]:
def duplicados_columnas(df):
    total_filas = len(df)
    resultados = {} # diccionario vacío para almacenar resultados con pares clave(nombre columna): valor(otro dict con los resultados 'v-unicos:100, etc)

    for columna in df.columns:
        conteo_repeticiones = df_history[columna].duplicated().sum()
        conteo_unicos = total_filas - conteo_repeticiones
        porcentaje_rep = (conteo_repeticiones / total_filas) * 100

        resultados[columna] = {
            'valores únicos': conteo_unicos,
            'valores repetidos': conteo_repeticiones,
            'repeticiones en porcentaje': f'{porcentaje_rep:.2f}%' # esta sintaxis: prefijo f-string indica a python que lo que le sigue debe tratarse como cadena formateada(permite usar variables y expresiones dentro de las llaves)
        }                                                          # .2f (.2)pide que se muestren dos decimales tras el punto, la f trata la variable como un float
    return pd.DataFrame(resultados).T

resultados_conteos = duplicados_columnas(df_history) # la ejecución de la fubnción, pasando a la variable temporal 'df' el df que queremos analizar
print(resultados_conteos)


## Limpieza dataset pre-mergeo:

In [ ]:
# Gestión registro anómalo, salario negativo. Convertir a nulo, para agruparlo con el resto de nulos de la columna y gestionarlos a posteriori juntos.

df_history.loc[df_history['Salary'] <=0, 'Salary'] = np.nan 
# método loc se usa para acceder a las filas de la columna a través de la condición que crea la máscara booleana
# Selecciona todas las filas donde el valor de 'Salary' sea 0 o menos de 0(negativa). 
# np.nan asigna el valor not a number a las celdas que cumplan la condición.

In [ ]:
# 2. Creación de una columna binaria, por asignación directa, que permita eliminar las columnas de Year y Month que tienen unos tantos por cientos elevadísimos de nulos.
# Sustituir esas dos columnas que no aportan información de calidad (es información escasa e irrelevante) por una sola columna que conteste realmente a la pregunta de si la suscripción del cliente está activa o no.
# Ésto redunda en la eficiencia y rapidez del proceso de unión.

df_history['Status'] = df_history['Cancellation Year'].notna().astype(int)  
# uso la columna cancellation year, en vez de la de month porque es más fiable (tiene 12 valores únicos, no como la de cancellation month que tiene 13: 12 meses+ NaN)

In [ ]:
# Eliminación columnas innecesarias:
df_history.drop(columns=['Cancellation Year', 'Cancellation Month'], inplace=True) # a columns=[] le pasas la lista de columnas que quieres borrar
# con inplace=True indicas que la eliminación es en el df original, sin crear una copia que haya que reasignar a posteriori.


In [ ]:
# Eliminación columnas innecesarias (si quisiéramos sacar el país de la clientela, que es por defecto, Canadá, podríamos usar las columnas de provincia, ciudad y código postal)
df_history.drop(columns= ['Country'], inplace=True)

# o tb podría ser esta sintaxis: df_history = df_history.drop(columns=['Country']) ésto genera una copia del df sin la columna dropeada, ocupa más espacio en memoria.

### Unión datasets:

In [ ]:
# Uso merge() por ek tipo de unión que hay que hacer, que es una fusión horizontal, fusionando los df's usando una columna común (como un JOIN de SQL)
# el concat() que es el otro método de combinar df, sería para una concatenación en vertical, apilando unas filas sobre otras (pero para ésto deberían de enter las mismas columnas)

df_fusionado = pd.merge(
    df_history,  # df_history es la tabla principal que contiene 1 registro único por cliente (sería como la PK)
    df_activity,  # df_activity contiene múltiples registros por cliente (y loyalty number sería su FK)
    on='Loyalty Number', # columna de unión
    how='left' # mantiene tabla de la izq (df_history) y agrega las coincidentes de la derecha. 
)

In [ ]:
# Repetir comprobación nulos, post mergeo:
df_fusionado.isnull().sum()

In [ ]:
# El número 103,152 es, por lo tanto, la cifra agregada del problema de nulos originales, expandida por el registro de actividad de cada cliente.
# La lógica es una simple multiplicación que confirma que la unión se realizó correctamente

In [ ]:
df_fusionado['Salary'].fillna(median_salary, inplace=True) #esto devuelve un future warning, que avisa de que aunque la asignación encadenada ahora funciona, en un futuro puede dejar de hacerlo
# vamos a usar otro método

In [ ]:
# NO EJECUTAR AÚN! 
# FASE DE VISUALIZACIONES (pegar esto cada vez que haga una gráfica):

# código para pegar al crear gráfica y que se ejecuten conjuntamente, para archivar directamente la imagen.
plt.figure(figsize=(10,6))
sns.barplot(data=df, x="Month", y="Flights Booked")
plt.title("Vuelos por mes")

plt.plot(x, y)
archivar_grafica("grafica_ejemplo")
plt.show()